There are multiple questions that are going in my head.

1. does win rate change over time? why?

2. Nasir club was the champion of the 2018/2019 season, Al hilal is the 2019/2020 season. What makes a champ?

3. does formation matter?

4. how important is the coach?

5. does the main scorrer participation in small clubs make as much difference as big clubs?

6. does playing in small unpopular venues make any difference?

7. how does previous match rating affect the next match result?

8. What is more relative, last matche rating, or the last match against the opposing team rating?

9. Does playing in unpopular venues have an effect on peformance?



In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

np.random.seed(42)
%matplotlib inline

In [128]:
season2018 = pd.read_csv("Season2018.csv")
season2019 = pd.read_csv("Season2019.csv")
season2020 = pd.read_csv("Season2020.csv")

df = pd.concat([season2018, season2019, season2020]).reset_index(drop = True)

In [129]:
df.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,151695,Al-Ettifaq,Al-Raed,1.0,1.0,Prince Mohamed Bin Fahd Stadium,NaN,2018-08-30T18:40:00+03:00,2018/2019,L. Ramos,B. Hasi,4-2-3-1,4-2-3-1,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...","[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",1,Match Finished,7.00,6.98
1,151696,Al-Hazm,Al Wehda Club,0.0,0.0,King Abdul Aziz Stadium,NaN,2018-08-30T19:05:00+03:00,2018/2019,D. Isăilă,Fábio Carille,4-2-3-1,4-4-2,"[('M. Asselah', None), ('Alemão', None), ('Kha...","[('Mohamed Awad', None), ('Osama Hawsawi', Non...",1,Match Finished,6.97,6.77
2,151697,Ohod,Al-Nassr,1.0,2.0,Prince Mohammed bin Abdul Aziz Stadium,NaN,2018-08-30T19:15:00+03:00,2018/2019,F. Arce,D. Carreño,4-2-3-1,4-2-3-1,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...","[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,Match Finished,6.64,6.83
3,151698,Al-Qadisiyah FC,Al-Fateh,0.0,0.0,Prince Saud bin Jalawi Stadium,"Mark Clattenburg, England",2018-08-31T18:40:00+03:00,2018/2019,A. Stanojević,F. Al Jabal,4-4-2,4-2-3-1,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...","[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",1,Match Finished,6.96,7.01
4,151699,Al-Hilal Saudi FC,Al-Fayha,1.0,0.0,King Fahd Stadium,NaN,2018-08-31T18:55:00+03:00,2018/2019,Jorge Jesus,G. Costas,4-1-4-1,4-3-3,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...","[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",1,Match Finished,7.07,6.55


let's first isolate a portion of the matches (test set) so that it doesn't affect our decision making in the exploration phase

* I'll isolate 20% of the data.
* and i'll isolate it so that we have 20% of each season in our test set
* I'll not use indices i'll use fixture_ids


In [130]:
print(df.shape)
print(season2018.shape)
print(season2019.shape)
print(season2020.shape)

(720, 19)
(240, 19)
(240, 19)
(240, 19)


In [131]:
np.random.seed(42)
season2018_test = np.random.permutation(season2018["fixture_id"])[:48]
season2019_test = np.random.permutation(season2019["fixture_id"])[:48]
season2020_test = np.random.permutation(season2020["fixture_id"])[:48]
test_fixtures = np.append(np.append(season2018_test, season2019_test), season2020_test)

In [132]:
len(test_fixtures)

144

In [133]:
test_fixtures[:10]

array([151719, 151701, 151790, 151805, 151800, 151867, 151929, 151783,
       151704, 151838])

In [134]:
test_set = df.loc[df["fixture_id"].isin(test_fixtures)]

In [135]:
len(test_set)

144

In [136]:
df = df.loc[~(df["fixture_id"].isin(test_fixtures))]

In [137]:
len(df)

576

# Initial data exploration

finding tredns & insights, and identifying flaws.

In [101]:
df.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,151695,Al-Ettifaq,Al-Raed,1.0,1.0,Prince Mohamed Bin Fahd Stadium,NaN,2018-08-30 18:40:00,2018/2019,L. Ramos,B. Hasi,4-2-3-1,4-2-3-1,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...","[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",1,Match Finished,7.00,6.98
1,151696,Al-Hazm,Al Wehda Club,0.0,0.0,King Abdul Aziz Stadium,NaN,2018-08-30 19:05:00,2018/2019,D. Isăilă,Fábio Carille,4-2-3-1,4-4-2,"[('M. Asselah', None), ('Alemão', None), ('Kha...","[('Mohamed Awad', None), ('Osama Hawsawi', Non...",1,Match Finished,6.97,6.77
2,151697,Ohod,Al-Nassr,1.0,2.0,Prince Mohammed bin Abdul Aziz Stadium,NaN,2018-08-30 19:15:00,2018/2019,F. Arce,D. Carreño,4-2-3-1,4-2-3-1,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...","[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,Match Finished,6.64,6.83
3,151698,Al-Qadisiyah FC,Al-Fateh,0.0,0.0,Prince Saud bin Jalawi Stadium,"Mark Clattenburg, England",2018-08-31 18:40:00,2018/2019,A. Stanojević,F. Al Jabal,4-4-2,4-2-3-1,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...","[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",1,Match Finished,6.96,7.01
4,151699,Al-Hilal Saudi FC,Al-Fayha,1.0,0.0,King Fahd Stadium,NaN,2018-08-31 18:55:00,2018/2019,Jorge Jesus,G. Costas,4-1-4-1,4-3-3,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...","[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",1,Match Finished,7.07,6.55


###### Let's start with the most basic operations

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 719
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fixture_id      576 non-null    int64  
 1   home_team       576 non-null    object 
 2   away_team       576 non-null    object 
 3   home_goals      558 non-null    float64
 4   away_goals      558 non-null    float64
 5   venue           576 non-null    object 
 6   referee         513 non-null    object 
 7   date            576 non-null    object 
 8   season          576 non-null    object 
 9   home_coach      531 non-null    object 
 10  away_coach      533 non-null    object 
 11  home_formation  545 non-null    object 
 12  away_formation  545 non-null    object 
 13  home_players    558 non-null    object 
 14  away_players    558 non-null    object 
 15  round           576 non-null    int64  
 16  status          576 non-null    object 
 17  H_avg_ratings   479 non-null    flo

* date column must be converted to datetime

In [97]:
df.describe()

,fixture_id,home_goals,away_goals,round,H_avg_ratings,A_avg_ratings
count,576.000000,558.000000,558.000000,576.000000,479.000000,479.000000
mean,357680.144097,1.553763,1.370968,15.432292,6.865261,6.819833
std,211801.638593,1.256857,1.183527,8.671328,0.230950,0.234582
min,151695.000000,0.000000,0.000000,1.000000,6.100000,6.150000
25%,151874.750000,1.000000,1.000000,8.000000,6.720000,6.675000
50%,312858.500000,1.000000,1.000000,15.000000,6.870000,6.830000
75%,635921.250000,2.000000,2.000000,23.000000,7.020000,6.980000
max,657882.000000,7.000000,6.000000,30.000000,7.500000,7.570000


we notice that the mean of the home team ratings is slightly greater than the away team which might mean that being home or not is a good predictor

In [146]:
df.date

0     2018-08-30 18:40:00
1     2018-08-30 19:05:00
2     2018-08-30 19:15:00
3     2018-08-31 18:40:00
4     2018-08-31 18:55:00
              ...        
714   2021-05-30 21:00:00
715   2021-05-30 21:00:00
716   2021-05-30 21:00:00
718   2021-05-30 21:00:00
719   2021-05-30 21:00:00
Name: date, Length: 576, dtype: datetime64[ns]

In [138]:
#Let's first do the perfrom basic operations 
df.isnull().sum()

fixture_id         0
home_team          0
away_team          0
home_goals        13
away_goals        13
venue              0
referee           62
date               0
season             0
home_coach        41
away_coach        39
home_formation    26
away_formation    26
home_players      13
away_players      13
round              0
status             0
H_avg_ratings     26
A_avg_ratings     26
dtype: int64

since only 13 matches are 'Uncollected yet' let's subtract 13 from the results. Negative entries mean no missing data

In [139]:
df.isnull().sum() - 13

fixture_id       -13
home_team        -13
away_team        -13
home_goals         0
away_goals         0
venue            -13
referee           49
date             -13
season           -13
home_coach        28
away_coach        26
home_formation    13
away_formation    13
home_players       0
away_players       0
round            -13
status           -13
H_avg_ratings     13
A_avg_ratings     13
dtype: int64

most missing values come from the referee column

* We have 49 missing referees, 54 coaches if you add home & away teams. 13 formations. 13 ratings

We can manually get the referee, coaches, formations. But sadly the ratings can only be collected through the api, and the api doesn't have this info yet, so we might look for another way to deal with this column.

In [155]:
null_data = df[df.isnull().any(axis=1)]

In [159]:
null_data.loc[null_data["H_avg_ratings"].isnull() == True]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
426,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,NaN,2020-08-09 19:30:00,2019/2020,D. Carreño,B. Hasi,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
439,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,P. Kralovec,2020-08-15 20:50:00,2019/2020,Rui Vitória,D. Carreño,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
453,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,NaN,2020-08-25 19:05:00,2019/2020,N. Zekri,V. Milojević,NaN,NaN,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
454,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,NaN,2020-08-25 19:20:00,2019/2020,Essa Al Mehyani,Vítor Campelos,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
455,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,A. Treimanis,2020-08-25 20:50:00,2019/2020,Rui Vitória,G. Solinas,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Abdul...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",27,Match Finished,NaN,NaN
461,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,João Pedro Pinheiro,2020-08-30 18:55:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,NaN,NaN,"[('Mustafa Malayekah', None), ('Igor Rossi', N...","[('Abdullah Al Jadani', None), ('Botía', None)...",28,Match Finished,NaN,NaN
464,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,Khaled Al Teris,2020-09-04 20:35:00,2019/2020,A. Chebbi,V. Milojević,NaN,NaN,"[(""A. M'Hamdi"", None), ('Osama Ashoor', None),...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",29,Match Finished,NaN,NaN
465,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,M. Conger,2020-09-04 20:35:00,2019/2020,Jorge Simão,G. Solinas,NaN,NaN,"[('Moslem Freej', None), ('Nawaf Al Sabhi', No...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",29,Match Finished,NaN,NaN
466,312959,Al-Ettifaq,Dhamk,0.0,2.0,Prince Mohamed bin Fahd Stadium,Majed Al Shamrani,2020-09-04 20:35:00,2019/2020,Khaled Al Atawi,N. Zekri,NaN,NaN,"[(""R. M'Bolhi"", None), ('Saleh Al Qumayzi', No...","[('M. Zeghba', None), ('S. Vittor', None), ('M...",29,Match Finished,NaN,NaN
467,312960,Al-Ittihad FC,Al-Nassr,1.0,1.0,King Abdullah Sports City,I. Bebek,2020-09-04 20:35:00,2019/2020,Fábio Carille,Rui Vitória,NaN,NaN,"[('Marcelo Grohe', None), ('Bruno Uvini', None...","[('B. Jones', None), ('Maicon', None), ('Abdul...",29,Match Finished,NaN,NaN


* we notice that when the formation column is missing, the ratings are also missing. and this data shortage only happend in the 2019/2020 season. the current season's ratings & formation's are all present which is a good indicator. Future matches might not encounter such absence of data.

to possibly change the structure of the data run the following cells

### Little discussion

In [110]:
df.columns

Index(['fixture_id', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'venue', 'referee', 'date', 'season', 'home_coach', 'away_coach',
       'home_formation', 'away_formation', 'home_players', 'away_players',
       'round', 'status', 'H_avg_ratings', 'A_avg_ratings'],
      dtype='object')

In order to make the data 'tidy' we must look at the home_team and away team columns, we know that each variable must have its own column. but these two columns violate this rule

This is not the optimal format for the data.
Maybe we should change the format so that we have the following 
* [Team, IsHome, opponent, Vinue, coach, opponent coach, formation, opponnet formation, average ratings, opponent ratings, Date, Season, referee, Goals Scored]
where each match is reperesnted by two rows.

Now the target variable is goals scored.
we can try to predict the final outcome of the match by running the model twice on both teams, then by comparing the goals we give the final result as Win, defeat, draw. 

* let's clean the data first and then apply this change.

# Data Cleaning 

* Well first try to fill in the missing data that we can using google's serach. It might be time consuming but complete data is always a plus.


* The referee column shouldn't logically be a predictor, as each referee should maintain neutrality. and not be in the favor of anytime. If used to predict it would just add noise to the model.
It also has many missing values so I'll prefer to drop the hall column.

#### Referee

In [163]:
df.drop("referee", axis = 1, inplace=True)

##### Coaches

In [164]:
df.loc[((df["home_coach"].isnull() == True) | (df["away_coach"].isnull() == True)) & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
480,635867,Al Shabab,Abha,1.0,0.0,Prince Faisal bin Fahd Stadium,2020-10-17 18:00:00,2020/2021,NaN,A. Chebbi,4-4-2,4-2-3-1,"[('Giedrius Arlauskis', 'G'), ('Mohammed Salem...","[('Abdelali Mhamdi', 'G'), ('Saeed Ali', 'D'),...",1,Match Finished,7.12,6.88
482,635869,Al Wehda Club,Al-Qadisiyah FC,2.0,1.0,King Abdulaziz Sports City Stadium,2020-10-17 20:00:00,2020/2021,NaN,NaN,4-4-2,4-3-2-1,"[('Abdullah Hussein Al Owisher', 'G'), ('Hamad...","[('Faisal Al-Masrahi', 'G'), ('Hamad Al Tohaif...",1,Match Finished,6.89,6.54
483,635870,Al-Hilal Saudi FC,Al-Ain,1.0,0.0,Prince Faisal bin Fahd Stadium,2020-10-17 21:15:00,2020/2021,NaN,NaN,4-4-2,4-2-3-1,"[('Abdullah Al-Mayouf', 'G'), ('Amiri Kurdi', ...","[('Mohammed Mazyad Alshammari', 'G'), ('Hassan...",1,Match Finished,7.08,6.77
493,635880,Al-Faisaly FC,Al Baten,2.0,1.0,Al Majma'ah Sports City Stadium,2020-10-23 19:55:00,2020/2021,NaN,NaN,4-3-3,4-3-2-1,"[('Mustafa Malaika', 'G'), ('Khaled Daghriri',...","[('Mazyad Al Enazi', 'G'), ('Saad Al Khayri', ...",2,Match Finished,6.95,6.54
495,635882,Al-Fateh,Al-Ittihad FC,2.0,2.0,Prince Abdullah bin Jalawi Sports City Stadium,2020-10-24 19:55:00,2020/2021,NaN,NaN,4-4-2,4-4-2,"[('Maksym Koval', 'G'), ('Nawaf Boushal', 'D')...","[('Marcelo Grohe', 'G'), ('Saud Abdulhamid', '...",2,Match Finished,6.81,6.90
499,635886,Al Wehda Club,Al-Faisaly FC,1.0,2.0,King Abdulaziz Sports City Stadium,2020-10-30 18:50:00,2020/2021,NaN,NaN,4-2-3-1,4-3-3,"[('Abdullah Hussein Al Owisher', 'G'), ('Ali A...","[('Ahmed Al-Kassar', 'G'), ('Khaled Daghriri',...",3,Match Finished,6.66,6.93
502,635889,Al-Ittihad FC,Al-Ahli Jeddah,2.0,0.0,King Abdullah Sports City,2020-10-31 18:50:00,2020/2021,NaN,NaN,4-4-2,4-2-3-1,"[('Marcelo Grohe', 'G'), ('Saud Abdulhamid', '...","[('Yasser Al-Mosailem', 'G'), ('Hassoun', 'D')...",3,Match Finished,6.99,6.68
504,635891,Al Wehda Club,Al-Raed,2.0,3.0,King Abdulaziz Sports City Stadium,2020-11-05 18:45:00,2020/2021,NaN,NaN,4-2-3-1,3-5-2,"[('Abdullah Hussein Al Owisher', 'G'), ('Ali A...","[('Azzedine Doukha', 'G'), ('Nemanja Miletić',...",4,Match Finished,6.66,6.73
508,635895,Al-Ittihad FC,Al Taawon,1.0,1.0,King Abdullah Sports City,2020-11-06 18:45:00,2020/2021,NaN,NaN,4-4-2,4-2-3-1,"[('Marcelo Grohe', 'G'), ('Saud Abdulhamid', '...","[('Hussein Shaian', 'G'), ('Yaseen Barnawi', '...",4,Match Finished,6.73,6.66
509,635896,Al-Ain,Al-Ahli Jeddah,3.0,4.0,King Saud Sport City Stadium,2020-11-07 14:35:00,2020/2021,NaN,V. Milojević,4-2-3-1,4-2-3-1,"[('Ameen Bokhari', 'G'), ('Mohammed Al-Shoraim...","[('Yasser Al-Mosailem', 'G'), ('Hassoun', 'D')...",4,Match Finished,6.67,6.89


In [175]:
df.loc[df["home_team"] == "Al-Qadisiyah FC"].home_coach.unique()

array(['A. Stanojević', 'I. Petev', 'Bandar Basryh', 'Y. Al Mannai'],
      dtype=object)

In [176]:
df.loc[df["home_team"] == "Al-Hilal Saudi FC"].home_coach.unique()

array(['Jorge Jesus', 'Z. Mamić', 'Péricles Chamusca', 'R. Lucescu', nan,
       'Rogério Micale', 'José Morais'], dtype=object)

In [177]:
df.loc[df["home_team"] == "Al-Ain"].home_coach.unique()

array(['M. Skibbe', nan, 'Pablo Machín'], dtype=object)

In [178]:
df.loc[df["home_team"] == "Al Baten"].home_coach.unique()

array(['F. Vercauteren', 'Yousef Alghadeer', 'M. Panait', 'José Garrido',
       'A. Veselinović', nan], dtype=object)

In [179]:
df.loc[df["home_team"] == "Al-Faisaly FC"].home_coach.unique()

array(['M. Rednic', 'F. Al-Waraqah', 'Péricles Chamusca',
       'Al Humaidy Al Otaibi', nan], dtype=object)

In [180]:
df.loc[df["home_team"] == "Al-Fateh"].home_coach.unique()

array(['F. Al Jabal', 'Y. Ferrera', nan], dtype=object)

In [181]:
df.loc[df["home_team"] == "Al-Ittihad FC"].home_coach.unique()

array(['Bandar Basryh', 'S. Bilić', 'J. Sierra', 'Mohammed Al-Abdali',
       'H. ten Cate', 'Fábio Carille', nan], dtype=object)

In [182]:
df.loc[df["home_team"] == "Al Wehda Club"].home_coach.unique()

array(['Fábio Carille', 'Mido', 'M. Cvitanović', 'D. Carreño',
       'Essa Al Mehyani', nan, 'Ivo Vieira', 'Mahmoud Al-Hadid',
       'G. Donis'], dtype=object)

In [183]:
df.loc[df["home_team"] == "Al-Ahli Jeddah"].home_coach.unique()

array(['P. Guede', 'J. Fossati', 'Youssef Anbar', 'B. Ivanković',
       'Saleh Al Mohammadi', 'C. Gross', 'Mazen Bahkali', 'V. Milojević',
       nan, 'F. Gormi', 'L. Reghecampf'], dtype=object)

In [185]:
df.loc[df["home_team"] == "Al-Raed"].home_coach.unique()

array(['B. Hasi', nan], dtype=object)

In [189]:
df.loc[df["away_team"] == "Al-Nassr"].away_coach.unique()

array(['D. Carreño', 'Hélder Cristóvão', 'Rui Vitória', 'A. Horvat',
       'Mano Menezes', nan], dtype=object)

In [190]:
df.loc[df["away_team"] == "Al Shabab"].away_coach.unique()

array(['M. Şumudică', 'J. Almirón', 'Luis García', 'Pedro Caixinha', nan,
       'Carlos Hernandez'], dtype=object)

In [192]:
df.loc[df["away_team"] == "Abha"].away_coach.unique()

array(['A. Chebbi', nan], dtype=object)

In [193]:
df.loc[df["away_team"] == "Al Taawon"].away_coach.unique()

array(['Pedro Emanuel', 'Paulo Sérgio', 'Vítor Campelos',
       'Abdullah Assiri', nan], dtype=object)

In [194]:
df.loc[df["away_team"] == "Al-Ettifaq"].away_coach.unique()


array(['L. Ramos', 'Sergio Piernas', 'Hélder Cristóvão',
       'Khaled Al Atawi', nan], dtype=object)

In [195]:
df.loc[df["away_team"] == "Al-Faisaly FC"].away_coach.unique()


array(['M. Rednic', 'Péricles Chamusca', nan], dtype=object)

In [196]:
df.loc[480, ["home_coach", "away_coach"]] = ['Pedro Caixinha', "A. Chebbi"]
df.loc[482, ["home_coach", "away_coach"]] = ['Ivo Vieira', 'Y. Al Mannai']
df.loc[483, ["home_coach", "away_coach"]] = ["R. Lucescu", 'M. Skibbe']
df.loc[493, ["home_coach", "away_coach"]] = ["Péricles Chamusca", 'José Garrido']
df.loc[495, ["home_coach", "away_coach"]] = ["Y. Ferrera", 'Fábio Carille']
df.loc[499, ["home_coach", "away_coach"]] = ['Ivo Vieira', "Péricles Chamusca"]
df.loc[502, ["home_coach", "away_coach"]] = ['Fábio Carille', 'V. Milojević']
df.loc[504, ["home_coach", "away_coach"]] = ['Ivo Vieira', 'B. Hasi']
df.loc[508, ["home_coach", "away_coach"]] = ['Fábio Carille', "Patrice Carteron"]
df.loc[509, ["home_coach", "away_coach"]] = ['M. Skibbe', 'V. Milojević']
df.loc[521, ["home_coach", "away_coach"]] = ['Fábio Carille', 'Y. Al Mannai']
df.loc[532, ["home_coach", "away_coach"]] = ['Ivo Vieira', 'Fábio Carille']
df.loc[542, ["home_coach", "away_coach"]] = ['Rui Vitória', 'V. Milojević']
df.loc[562, ["home_coach", "away_coach"]] = ["R. Lucescu", 'Pedro Caixinha']
df.loc[580, ["home_coach", "away_coach"]] = ['Fábio Carille', 'A. Chebbi']
df.loc[605, ["home_coach", "away_coach"]] = ['V. Milojević', 'José Garrido']
df.loc[609, ["home_coach", "away_coach"]] = ['A. Horvat', "Patrice Carteron"]
df.loc[617, ["home_coach", "away_coach"]] = ['V. Milojević', 'Fábio Carille']
df.loc[622, ["home_coach", "away_coach"]] = ['A. Horvat', 'Carlos Hernandez']
df.loc[627, ["home_coach", "away_coach"]] = ['V. Milojević', 'Pablo Machín']
df.loc[631, ["home_coach", "away_coach"]] = ['Rogério Micale', 'Khaled Al Atawi']
df.loc[639, ["home_coach", "away_coach"]] = ['A. Horvat', 'Rogério Micale']
df.loc[642, ["home_coach", "away_coach"]] = ['Péricles Chamusca', 'V. Milojević']
df.loc[646, ["home_coach", "away_coach"]] = ['A. Horvat', 'A. Chebbi']
df.loc[654, ["home_coach", "away_coach"]] = ['Fábio Carille', 'Mahmoud Al-Hadid']
df.loc[659, ["home_coach", "away_coach"]] = ['Péricles Chamusca', 'B. Hasi']
df.loc[660, ["home_coach", "away_coach"]] = ['Fábio Carille', 'Carlos Hernandez']
df.loc[699, ["home_coach", "away_coach"]] = ['José Morais', "'L. Reghecampf'"]

In [197]:
df.loc[((df["home_coach"].isnull() == True) | (df["away_coach"].isnull() == True)) & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings


* Now we have all our coaches present 

##### Formation column

In [198]:
df.loc[((df["home_formation"].isnull() == True) | (df["away_formation"].isnull() == True)) & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
426,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,2020-08-09 19:30:00,2019/2020,D. Carreño,B. Hasi,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
439,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,2020-08-15 20:50:00,2019/2020,Rui Vitória,D. Carreño,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
453,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,2020-08-25 19:05:00,2019/2020,N. Zekri,V. Milojević,NaN,NaN,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
454,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,2020-08-25 19:20:00,2019/2020,Essa Al Mehyani,Vítor Campelos,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
455,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,2020-08-25 20:50:00,2019/2020,Rui Vitória,G. Solinas,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Abdul...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",27,Match Finished,NaN,NaN
461,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,2020-08-30 18:55:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,NaN,NaN,"[('Mustafa Malayekah', None), ('Igor Rossi', N...","[('Abdullah Al Jadani', None), ('Botía', None)...",28,Match Finished,NaN,NaN
464,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,2020-09-04 20:35:00,2019/2020,A. Chebbi,V. Milojević,NaN,NaN,"[(""A. M'Hamdi"", None), ('Osama Ashoor', None),...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",29,Match Finished,NaN,NaN
465,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,2020-09-04 20:35:00,2019/2020,Jorge Simão,G. Solinas,NaN,NaN,"[('Moslem Freej', None), ('Nawaf Al Sabhi', No...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",29,Match Finished,NaN,NaN
466,312959,Al-Ettifaq,Dhamk,0.0,2.0,Prince Mohamed bin Fahd Stadium,2020-09-04 20:35:00,2019/2020,Khaled Al Atawi,N. Zekri,NaN,NaN,"[(""R. M'Bolhi"", None), ('Saleh Al Qumayzi', No...","[('M. Zeghba', None), ('S. Vittor', None), ('M...",29,Match Finished,NaN,NaN
467,312960,Al-Ittihad FC,Al-Nassr,1.0,1.0,King Abdullah Sports City,2020-09-04 20:35:00,2019/2020,Fábio Carille,Rui Vitória,NaN,NaN,"[('Marcelo Grohe', None), ('Bruno Uvini', None...","[('B. Jones', None), ('Maicon', None), ('Abdul...",29,Match Finished,NaN,NaN


##### Fixing the date column

In [140]:
df.loc[0, "date"]

'2018-08-30T18:40:00+03:00'

In [141]:
df["date"] = [date[:-6].replace("T", " ") for date in df["date"]]

In [142]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")


In [145]:
df.date.dtype

dtype('<M8[ns]')

# Data Wrangling 

###### changing the strucutre of the data so that we can perform better visualizations and predictions

In [ ]:
# since every entry now is based on home I'll create a feature called IsHome and set it to 1

df["IsHome"] = 1

In [ ]:
len([x for x in range(0, len(df)*2, 2)])

In [ ]:
len(df)

In [ ]:
df.index = [x for x in range(0, len(df) * 2, 2)]

In [ ]:
df.columns

I'll change the order of the columns so that it is close to the desired format

In [ ]:
df = df[["home", "home_format", "IsHome", "away", "away_format", "players","referee", "season", "home_goals", "away_goals"]]

In [ ]:
for i in [x for x in range(1, len(df) * 2 + 1, 2)]:
    old_entry = df.loc[i-1]
    df.loc[i] = [old_entry.away, old_entry.away_format, 0, old_entry.home, old_entry.home_format, old_entry.players, old_entry.referee, old_entry.season, old_entry.away_goals, np.nan]
    

In [ ]:
df.sort_index(inplace=True)

In [ ]:
# Now we will change the columns so that it aligns with our format

df.drop("away_goals", inplace=True, axis=1)

df.columns = ["Team1", "format", "IsHome", "Opponent", "Opponent_format", "Players", "Referee", "Season", "GoalsScored"]

In [ ]:
df.GoalsScored = df.GoalsScored.astype("int64")

# Feature Engineering

In [ ]:
df["pts"] = 0
df["opponent_pts"] = 0
df["result"] = 0

In [105]:
#Adding the current points feature. Assuming change of structure has occured

for season in ["2018/2019", "2019/2020", "2020/2021"]:
    
    season_df = df.loc[(df["Season"] == season )]

    points = {}

    for team in season_df.home_team.unique():
    points[team] = 0
    
    
    for i in season_df.index:
        if i % 2 == 0 :
            Team1 = season_df.loc[i, "Team1"]
            Team2 = season_df.loc[i, "Opponent"]


            season_df.loc[i,"pts"] = points[Team1]
            season_df.loc[i, "opponent_pts"] = points[Team1]

            season_df.loc[i+1,"pts"] = points[Team2]
            season_df.loc[i+1, "opponent_pts"] = points[Team1]


            if season_df.loc[i, "GoalsScored"] > season_df.loc[i+1, "GoalsScored"]:
                points[Team1] += 3
                season_df.loc[i, "result"] = "Win"
                season_df.loc[i+1, "result"] = "Defeat"
            
            elif season_df.loc[i, "GoalsScored"] == season_df.loc[i+1, "GoalsScored"]:
                points[Team1] += 1
                points[Team2] += 1
                season_df.loc[i, "result"] = "Draw"
                season_df.loc[i+1, "result"] = "Draw"

            else:
                points[Team2] += 3
                season_df.loc[i, "result"] = "Defeat"
                season_df.loc[i+1, "result"] = "Win"

        df.loc[df["Season"] == season] = season_df




In [106]:
# Adding the final state feature

In [107]:
# Adding the last game performance

In [108]:
# Adding the last game agains the other team performance

In [199]:
df.to_csv("all_seasons_with_coaches.csv", index=False)